# MissOh DataLoader

In [505]:
# authored by haeyong.kang
# date : 2020/06/21

### AnotherMissOh Visual Structure
- json_data['file_name'] : 'AnotherMissOh01.mp4'
- json_data['visual_results']
- json_data['visual_results'][0].keys() : dict_keys(['start_time', 'end_time', 'vid', 'image_info'])
- {
'start_time': '00:02:51;16', 
'end_time': '00:02:54;15', 
'vid': 'AnotherMissOh01_001_0078', 
'image_info': ...}
- json_data['visual_results'][0]['image_info']
- [{'frame_id': 'AnotherMissOh01_001_0078_IMAGE_0000004295', 
'place': 'none', 
'persons': [
{'person_id': 'Haeyoung1', 
'person_info': {
'face_rect': {'min_x': 515, 'min_y': 0, 'max_x': 845, 'max_y': 443}, 
'full_rect': {'min_x': 278, 'min_y': 2, 'max_x': 1025, 'max_y': 769}, 
'behavior': 'stand up', 
'predicate': 'none', 
'emotion': 'Neutral', 
'face_rect_score': '0.5', 
'full_rect_score': '0.9'}, 
'related_objects': []}], 
'objects': []}, 
- {'frame_id': 'AnotherMissOh01_001_0078_IMAGE_0000004311', 
'place': '', 
'persons': [{
'person_id':'Haeyoung1',
'person_info': {
'face_rect': {'min_x': 515, 'min_y': 0, 'max_x': 831, 'max_y': 411}, 
'full_rect': {'min_x': 270, 'min_y': 0, 'max_x': 1025, 'max_y': 768}, 
'behavior': 'stand up', 
'predicate': 'none', 
'emotion': 'Neutral', 
'face_rect_score': '0.5', 
'full_rect_score': '0.9'}, 
'related_objects': []}],
'objects': []},]

In [538]:
import sys
sys.path.append("../") # go to parent dir

In [539]:
import os
from torch.utils.data import Dataset, DataLoader
import cv2
import pickle
import numpy as np
import glob
from torchvision.transforms import Compose, Resize, ToTensor, Normalize
from PIL import Image
import json
import argparse
import matplotlib.pyplot as plt

from Yolo_v2_pytorch.src.utils import *
from graphviz import Digraph, Graph

import networkx as nx
from networkx.drawing.nx_pydot import read_dot
#from networkx.drawing.nx_agraph import read_dot
from networkx.readwrite import json_graph

In [540]:
read_dot

<function networkx.drawing.nx_pydot.read_dot(path)>

In [541]:
def is_not_blank(s):
    return bool(s and s.strip())

In [542]:
MissOh_CLASSES = ['person']
print(MissOh_CLASSES[0])
global colors
colors = pickle.load(open("../Yolo_v2_pytorch/src/pallete", "rb"))
print(colors[0])

person
(39, 129, 113)


In [543]:
def get_args():
    parser = argparse.ArgumentParser(
        "You Only Look Once:Unified, Real-Time Object Detection")
    parser.add_argument("--image_size", type=int,
                        default=448,
                        help="The common width and height for all images")
    parser.add_argument("--batch_size", type=int, default=1,
                        help="The number of images per batch")
    # Training base Setting
    parser.add_argument("--momentum", type=float, default=0.9)
    parser.add_argument("--decay", type=float, default=0.0005)
    parser.add_argument("--dropout", type=float, default=0.5)
    parser.add_argument("--num_epoches", type=int, default=100)
    parser.add_argument("--test_interval", type=int, default=1,
                        help="Number of epoches between testing phases")
    parser.add_argument("--object_scale", type=float, default=1.0)
    parser.add_argument("--noobject_scale", type=float, default=0.5)
    parser.add_argument("--class_scale", type=float, default=1.0)
    parser.add_argument("--coord_scale", type=float, default=5.0)
    parser.add_argument("--reduction", type=int, default=32)
    parser.add_argument("--es_min_delta", type=float, default=0.0,
                        help="Early stopping's parameter:minimum change loss to qualify as an improvement")
    parser.add_argument("--es_patience", type=int, default=0,
                        help="Early stopping's parameter:number of epochs with no improvement after which training will be stopped. Set to 0 to disable this technique.")

    parser.add_argument("--pre_trained_model_type",
                        type=str, choices=["model", "params"],
                        default="model")
    parser.add_argument("--pre_trained_model_path", type=str,
                        default="Yolo_v2_pytorch/trained_models/only_params_trained_yolo_voc") # Pre-training path

    parser.add_argument("--saved_path", type=str,
                        default="./checkpoint") # saved training path
    parser.add_argument("--conf_threshold", type=float, default=0.35)
    parser.add_argument("--nms_threshold", type=float, default=0.5)
    args = parser.parse_args(args=[]) # for jupyter 
    return args

In [544]:
opt = get_args()
print(opt)

Namespace(batch_size=1, class_scale=1.0, conf_threshold=0.35, coord_scale=5.0, decay=0.0005, dropout=0.5, es_min_delta=0.0, es_patience=0, image_size=448, momentum=0.9, nms_threshold=0.5, noobject_scale=0.5, num_epoches=100, object_scale=1.0, pre_trained_model_path='Yolo_v2_pytorch/trained_models/only_params_trained_yolo_voc', pre_trained_model_type='model', reduction=32, saved_path='./checkpoint', test_interval=1)


In [545]:
# visualize the images and labels
height, width = (768, 1024)
width_ratio = 448 / width
height_ratio = 448 / height

In [557]:
class AnotherMissOh(Dataset):
    def __init__(self, dataset, img_path, json_path, display_log=True):
        
        self.display_log = display_log
        self.init_clips(img_path)
        self.load_json(dataset,img_path, json_path)
        
    def init_clips(self, img_path):
        self.cnt_clips = 0
        self.img_path = img_path
        
        self.img_size = (1024, 768)
        self.img_scaled_size = (448, 448)
        
        tform = [
            Resize(self.img_scaled_size), # should match to Yolo_V2
            ToTensor(), 
            #Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) # should match to Yolo_V2
        ]
        self.transformations = Compose(tform)
        
        '''
        clips = {
            'episode' : [],
            'clip' : [],
            'start_time' : [],
            'end_time' : [],
            'vid' : [],
            'img_size' : [],
            'img_scaled_size' : [],
            'image_info' : []}
            
        image_info = {
            'frame_id': [],
            'place' : [],
            'persons' : []}
            
        persons = {
            'person_id': [],
            'face_rect' : [],
            'full_rect' : [],
            'behavior' : [],
            'predicate' : [],
            'emotion' : [],
            'face_rect_score' : [],
            'full_rect_score' : []}
        '''
        
    def load_json(self, dataset, img_path, json_path):
        
        self.clips = []
        
        for episode in dataset:
            img_dir = img_path + 'AnotherMissOh{:02}/'.format(episode)
            json_dir = json_path + 'AnotherMissOh{:02}_ver3.2.json'.format(episode)
            if self.display_log:
                print('imag_dir:{}'.format(img_dir))
                print('json_dir:{}'.format(json_dir))

            with open(json_dir, encoding='utf-8') as json_file:
                json_data = json.load(json_file)

            for i in range(len(json_data['visual_results'])):
                clip = {}
                clip['episode'] = []
                clip['clip'] = []
                clip['start_time'] = []
                clip['end_time'] = []
                clip['vid'] = []
                clip['image_info'] = []
                
                if self.display_log:
                    print("***{}th episode***{}th clips***************************************".format(episode, i))
                    print("['visual_results'][{}]['start_time']:{}".format(i,json_data['visual_results'][i]['start_time']))
                    print("['visual_results'][{}]['end_time']:{}".format(i,json_data['visual_results'][i]['end_time']))
                    print("['visual_results'][{}]['vid']:{}".format(i,json_data['visual_results'][i]['vid'].replace('_', '/')))
                    print("['visual_results'][{}]['img_size']:{}".format(i,img_size))
                    print("['visual_results'][{}]['img_scaled_size']:{}".format(i,img_scaled_size))
                    print("['visual_results'][{}]['episode']:{}".format(i,episode))
                
                clip['episode'].append(episode)
                clip['clip'].append(i)
                clip['start_time'].append(json_data['visual_results'][i]['start_time'])
                clip['end_time'].append(json_data['visual_results'][i]['end_time'])
                clip['vid'].append(json_data['visual_results'][i]['vid'].replace('_', '/'))
                
                
                for j, info in enumerate(json_data['visual_results'][i]['image_info']):
                    image_info = {}
                    image_info['frame_id'] = []
                    image_info['place'] = []
                    image_info['persons'] = {}
                    
                    if self.display_log:
                        print("=============={}th frame========================================".format(j))
                        
                    img_file = img_dir + json_data['visual_results'][i]['vid'].replace('_', '/')[-8:] + '/'+ info['frame_id'][-16:] + '.jpg'
                    image_info['frame_id'].append(img_file)
                    image_info['place'].append(info['place'])
                    
                    image_info['persons']['person_id']=[]
                    image_info['persons']['face_rect']=[]
                    image_info['persons']['full_rect']=[]
                    image_info['persons']['behavior']=[]
                    image_info['persons']['predicate']=[]
                    image_info['persons']['emotion']=[]
                    image_info['persons']['face_rect_score']=[]
                    image_info['persons']['full_rect_score']=[]
                    for k, person in enumerate(info['persons']):
                        if self.display_log:
                            print("--------------------{}th person-----------------------------".format(k))
                            
                        image_info['persons']['person_id'].append(person['person_id'])
                        face_bbox = person['person_info']['face_rect']
                        if (face_bbox['min_y'] == "" 
                            or face_bbox['max_y'] == "" 
                            or face_bbox['min_x'] == "" 
                            or face_bbox['max_x'] == ""):
                            face_rect = []
                            continue
                        else:
                            face_rect = [face_bbox['min_x'], face_bbox['min_y'], face_bbox['max_x'], face_bbox['max_y']]
                        image_info['persons']['face_rect'].append(face_rect)
                        full_bbox = person['person_info']['full_rect']
                        if (full_bbox['min_y'] == "" 
                            or full_bbox['max_y'] == "" 
                            or full_bbox['min_x'] == "" 
                            or full_bbox['max_x'] == ""):
                            full_rect = []
                            continue
                        else:
                            full_rect = [full_bbox['min_x'], full_bbox['min_y'], full_bbox['max_x'], full_bbox['max_y']]
                        image_info['persons']['full_rect'].append(full_rect)
                        image_info['persons']['behavior'].append(person['person_info']['behavior'])
                        image_info['persons']['predicate'].append(person['person_info']['predicate'])
                        image_info['persons']['emotion'].append(person['person_info']['emotion'])
                        image_info['persons']['face_rect_score'].append(person['person_info']['face_rect_score'])
                        image_info['persons']['full_rect_score'].append(person['person_info']['full_rect_score'])
                        
                    clip['image_info'].append(image_info)
                self.clips.append(clip)
                    
    def __len__(self):
        return len(self.clips)
                    
    def __getitem__(self, item):
        info = self.clips[item]['image_info']
        episode = self.clips[item]['episode']
        clip = self.clips[item]['clip']
        start_time = self.clips[item]['start_time']
        end_time = self.clips[item]['start_time']
        
        images = []
        for it, frame in enumerate(info):
            img = Image.open(frame['frame_id'][0]).convert('RGB')
            img = self.transformations(img)
            images.append(img)
        
        return images, info, episode, clip, start_time, end_time

In [558]:
img_path = '../data/AnotherMissOh/AnotherMissOh_images/'
json_path = '../data/AnotherMissOh/AnotherMissOh_Visual_ver3.2/'

episode = 1
train = [episode]
train_set = AnotherMissOh(train, img_path, json_path, False)

In [559]:
def graph(episode, scene, frm,st,et,info, save_file, debug = False):
    import string
    strseq = string.ascii_uppercase
    
    # define  graph
    dot = Digraph('G',filename='{}.gv'.format(save_file),engine='fdp')
    dot.attr('graph', rotate = '0', dpi='600',rankdir='TB', size='10,5')
    dot.attr('node', height='0.1', fontsize='6')
    dot.attr('edge', fontsize='1')

    place = "{}".format(info['place'][0])
    sound = "{}".format('sound')
    
    if not is_not_blank(place):
        place = 'none'
    if not is_not_blank(sound):
        sound = 'none'
        
    num_of_persons = len(info['persons']['person_id'])
    
    frm_graph = 'episode_{}_scene_{}_frame_{}'.format(
        episode, scene, frm)
    
    #dot.node(frm_graph, style='filled', color='lightgrey')
    episode_node = "episode_{:02d}".format(episode)
    scene_node = "scene_{:03d}".format(scene)
    frame_node = "frame_{:04d}".format(frm)
    dot.node(episode_node, style='filled', color='lightgrey')
    dot.node(scene_node, style='filled', color='lightgrey')
    dot.node(frame_node, style='filled', color='lightgrey')
    
    dot.node(place, style='filled', color='lightblue')
    dot.node(sound, style='filled', color='lightblue')
    
    if is_not_blank(episode_node) and is_not_blank(scene_node):
        dot.edge(episode_node, scene_node)
    
    if is_not_blank(scene_node) and is_not_blank(frame_node):
        dot.edge(scene_node, frame_node)
        
    if is_not_blank(frame_node) and is_not_blank(place):
        dot.edge(frame_node, place)
    
    if is_not_blank(frame_node) and is_not_blank(sound):
        dot.edge(frame_node, sound)

    for p in range(num_of_persons):
        
        try:
            person_id = "{}".format(info['persons']['person_id'][p])
        except:
            person_id = 'none'
            continue
        try:
            behavior = "{}".format(info['persons']['behavior'][p])
        except:
            person_id = 'none'
            continue
        try:
            predicate = "{}".format(info['persons']['predicate'][p])
        except:
            person_id = 'none'
            continue
        try:
            emotion = "{}".format(info['persons']['emotion'][p])
        except:
            person_id = 'none'
            continue

        if is_not_blank(person_id):
            dot.node(person_id)
        if is_not_blank(behavior):
            dot.node(behavior, style='filled', color='green')
        if is_not_blank(predicate):
            dot.node(predicate, style='filled', color='yellow')
        if is_not_blank(emotion):
            dot.node(emotion, style='filled', color='blue')

        if is_not_blank(frame_node) and is_not_blank(person_id):
            dot.edge(frame_node, person_id)
        if is_not_blank(person_id) and is_not_blank(behavior):
            dot.edge(person_id, behavior)
        if is_not_blank(person_id) and is_not_blank(predicate):
            dot.edge(person_id, predicate)
        if is_not_blank(person_id) and is_not_blank(emotion):
            dot.edge(person_id, emotion)
            
        
            
    # show in image
    dot.format = 'png'
    dot.render('{}.gv'.format(save_file), view=True)

    graph = cv2.imread('{}.gv.png'.format(save_file))
    graph = cv2.resize(graph, dsize=(0, 0), fx=600.0/graph.shape[0], fy=600.0/graph.shape[0])

    if debug:
        plt.figure(figsize=(8,8))
        plt.imshow(graph)
        plt.show()

In [560]:
save_dir = '../results/drama_graph/'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

In [ ]:
debug = False

for scene in range(len(train_set)):
    #if scene > 30:
    #    break
        
    images, info, episode, scene, st, et = train_set[scene]
    scene= scene[0]
    episode= episode[0]

    for frm in range(len(images)):
        image = images[frm].cpu().numpy()
        #print(image)
        imageInfo = cv2.cvtColor(np.transpose(image * 255,(1,2,0)), 
                                 cv2.COLOR_RGB2BGR)

        frm_name = "episode_{:02d}_scene_{:03d}_frame_{:04d}".format(episode, scene, frm)
        save_file = save_dir + frm_name
        print("episode:{}, scene:{}, frame:{} st:{}, et:{}".format(
            episode, scene, frm, st, et))

        place = info[frm]['place'][0]
        sound = 'sound'
        num_of_persons = len(info[frm]['persons']['person_id'])
        graph(episode, scene, frm, st, et ,info[frm], save_file, debug)

        # read dot graph
        dot_graph = nx.nx_pydot.read_dot('{}.gv'.format(save_file))
        s_graph = json.dumps(json_graph.node_link_data(dot_graph))

        for p in range(num_of_persons):
            try: 
                person_id = info[frm]['persons']['person_id'][p]
            except:
                person_id = 'none'
                continue
            try:
                face_rect= info[frm]['persons']['face_rect'][p]
            except:
                face_rect = 'none'
                continue
            try: 
                full_rect = info[frm]['persons']['full_rect'][p]
            except:
                full_rect = 'none'
                continue
            try: 
                behavior = info[frm]['persons']['behavior'][p]
            except:
                behavior = 'none'
                continue
            try:
                predicate = info[frm]['persons']['predicate'][p]
            except:
                predicate = 'none'
                continue
            try: 
                emotion = info[frm]['persons']['emotion'][p]
            except:
                emotion = 'none'
                continue

            print("place:{}".format(place))
            print("num_of_persons:{}".format(num_of_persons))
            print("person_id:{}".format(person_id))
            print("face_rect:{}".format(face_rect))
            print("full_rect:{}".format(full_rect))
            print("behavior:{}".format(behavior))
            print("predicate:{}".format(predicate))
            print("emotion:{}".format(emotion))

            # face rect
            xmin = int(max(face_rect[0] * width_ratio, 0))
            ymin = int(max(face_rect[1] * height_ratio, 0))
            xmax = int(min((face_rect[2]) * width_ratio, 448))
            ymax = int(min((face_rect[3]) * height_ratio, 448))
            cv2.rectangle(imageInfo, (xmin, ymin), (xmax, ymax), colors[0], 2)

            # full rect
            xmin = int(max(full_rect[0] * width_ratio, 0))
            ymin = int(max(full_rect[1] * height_ratio, 0))
            xmax = int(min((full_rect[2]) * width_ratio, 448))
            ymax = int(min((full_rect[3]) * height_ratio, 448))
            cv2.rectangle(imageInfo, (xmin, ymin), (xmax, ymax), colors[2], 2)
            cv2.putText(imageInfo, person_id, (xmin, ymin), 
                        cv2.FONT_HERSHEY_PLAIN, 1.5, (255.0, 255.0, 255.0), 1)
            cv2.putText(imageInfo, emotion, (xmin, ymax+20), 
                        cv2.FONT_HERSHEY_PLAIN, 1, (0.0, 0.0, 255.0), 1)
            cv2.putText(imageInfo, behavior, (xmin, ymax+30), 
                        cv2.FONT_HERSHEY_PLAIN, 1, (0.0, 255.0, 0.0), 1)
            cv2.putText(imageInfo, place, (10, 20), 
                        cv2.FONT_HERSHEY_PLAIN, 1.5, (255.0, 255.0, 255.0), 1)
            cv2.putText(imageInfo, sound, (10, 40), 
                        cv2.FONT_HERSHEY_PLAIN, 1.5, (255.0, 255.0, 255.0), 1)

        imageInfo = cv2.resize(imageInfo,dsize=(1024, 768))
        cv2.imwrite("{}_frame.png".format(save_file), imageInfo)
        imageInfo = cv2.cvtColor(imageInfo, cv2.COLOR_BGR2RGB)

        if debug:
            plt.figure(figsize=(8,8))
            plt.imshow(np.uint8(imageInfo))
            plt.show()

episode:1, scene:0, frame:0 st:['00:02:51;16'], et:['00:02:51;16']
place:none
num_of_persons:1
person_id:Haeyoung1
face_rect:[515, 0, 845, 443]
full_rect:[278, 2, 1025, 769]
behavior:stand up
predicate:none
emotion:Neutral
episode:1, scene:0, frame:1 st:['00:02:51;16'], et:['00:02:51;16']
place:
num_of_persons:1
person_id:Haeyoung1
face_rect:[515, 0, 831, 411]
full_rect:[270, 0, 1025, 768]
behavior:stand up
predicate:none
emotion:Neutral
episode:1, scene:0, frame:2 st:['00:02:51;16'], et:['00:02:51;16']
place:
num_of_persons:1
person_id:Haeyoung1
face_rect:[515, 0, 829, 442]
full_rect:[255, 0, 1022, 767]
behavior:stand up
predicate:none
emotion:Neutral
episode:1, scene:0, frame:3 st:['00:02:51;16'], et:['00:02:51;16']
place:
num_of_persons:1
person_id:Haeyoung1
face_rect:[514, 0, 836, 438]
full_rect:[255, 0, 1022, 767]
behavior:stand up
predicate:none
emotion:Neutral
episode:1, scene:0, frame:4 st:['00:02:51;16'], et:['00:02:51;16']
place:
num_of_persons:1
person_id:Haeyoung1
face_rect

place:kitchen
num_of_persons:1
person_id:Deogi
face_rect:[279, 107, 320, 169]
full_rect:[237, 76, 363, 605]
behavior:stand up
predicate:
emotion:Neutral
episode:1, scene:6, frame:5 st:['00:03:06;03'], et:['00:03:06;03']
episode:1, scene:6, frame:6 st:['00:03:06;03'], et:['00:03:06;03']
episode:1, scene:7, frame:0 st:['00:03:08;07'], et:['00:03:08;07']
episode:1, scene:7, frame:1 st:['00:03:08;07'], et:['00:03:08;07']
episode:1, scene:7, frame:2 st:['00:03:08;07'], et:['00:03:08;07']
episode:1, scene:8, frame:0 st:['00:03:09;05'], et:['00:03:09;05']
episode:1, scene:8, frame:1 st:['00:03:09;05'], et:['00:03:09;05']
episode:1, scene:8, frame:2 st:['00:03:09;05'], et:['00:03:09;05']
episode:1, scene:9, frame:0 st:['00:03:10;04'], et:['00:03:10;04']
place:livingroom
num_of_persons:1
person_id:Kyungsu
face_rect:[596, 326, 645, 405]
full_rect:[526, 290, 750, 661]
behavior:
predicate:
emotion:Neutral
episode:1, scene:9, frame:1 st:['00:03:10;04'], et:['00:03:10;04']
place:livingroom
num_of_pe

place:livingroom
num_of_persons:2
person_id:Jeongsuk
face_rect:[591, 180, 621, 225]
full_rect:[536, 154, 648, 318]
behavior:
predicate:
emotion:Neutral
place:livingroom
num_of_persons:2
person_id:Kyungsu
face_rect:[615, 392, 666, 457]
full_rect:[580, 326, 761, 595]
behavior:
predicate:
emotion:Neutral
episode:1, scene:12, frame:0 st:['00:03:20;12'], et:['00:03:20;12']
place:
num_of_persons:1
person_id:Deogi
face_rect:[368, 132, 552, 409]
full_rect:[156, 34, 836, 769]
behavior:none
predicate:
emotion:Happiness
episode:1, scene:12, frame:1 st:['00:03:20;12'], et:['00:03:20;12']
place:
num_of_persons:1
person_id:Deogi
face_rect:[378, 150, 573, 419]
full_rect:[149, 34, 836, 769]
behavior:none
predicate:
emotion:Happiness
episode:1, scene:12, frame:2 st:['00:03:20;12'], et:['00:03:20;12']
place:
num_of_persons:1
person_id:Deogi
face_rect:[448, 166, 675, 459]
full_rect:[174, 34, 861, 769]
behavior:none
predicate:
emotion:Happiness
episode:1, scene:12, frame:3 st:['00:03:20;12'], et:['00:03:2

episode:1, scene:17, frame:0 st:['00:03:32;00'], et:['00:03:32;00']
place:
num_of_persons:1
person_id:Haeyoung1
face_rect:[525, 23, 828, 443]
full_rect:[402, 0, 1000, 767]
behavior:none
predicate:
emotion:Neutral
episode:1, scene:17, frame:1 st:['00:03:32;00'], et:['00:03:32;00']
place:
num_of_persons:1
person_id:Haeyoung1
face_rect:[543, 30, 826, 443]
full_rect:[406, 0, 1009, 766]
behavior:none
predicate:
emotion:Neutral
episode:1, scene:17, frame:2 st:['00:03:32;00'], et:['00:03:32;00']
place:
num_of_persons:1
person_id:Haeyoung1
face_rect:[514, 13, 831, 447]
full_rect:[388, 0, 1007, 768]
behavior:none
predicate:
emotion:Neutral
episode:1, scene:17, frame:3 st:['00:03:32;00'], et:['00:03:32;00']
place:
num_of_persons:1
person_id:Haeyoung1
face_rect:[511, 19, 822, 439]
full_rect:[392, 0, 1007, 768]
behavior:none
predicate:
emotion:Neutral
episode:1, scene:17, frame:4 st:['00:03:32;00'], et:['00:03:32;00']
place:
num_of_persons:1
person_id:Haeyoung1
face_rect:[533, 59, 816, 465]
full_r

place:livingroom
num_of_persons:2
person_id:Haeyoung1
face_rect:[124, 286, 197, 387]
full_rect:[49, 199, 301, 607]
behavior:sit down
predicate:
emotion:Neutral
place:livingroom
num_of_persons:2
person_id:Deogi
face_rect:[781, 360, 840, 436]
full_rect:[760, 319, 896, 645]
behavior:
predicate:
emotion:Disgust
episode:1, scene:17, frame:30 st:['00:03:32;00'], et:['00:03:32;00']
place:livingroom
num_of_persons:1
person_id:Haeyoung1
face_rect:[125, 286, 198, 387]
full_rect:[49, 199, 301, 608]
behavior:sit down
predicate:
emotion:Neutral
episode:1, scene:17, frame:31 st:['00:03:32;00'], et:['00:03:32;00']
place:livingroom
num_of_persons:1
person_id:Haeyoung1
face_rect:[125, 286, 198, 387]
full_rect:[49, 199, 301, 608]
behavior:sit down
predicate:
emotion:Neutral
episode:1, scene:17, frame:32 st:['00:03:32;00'], et:['00:03:32;00']
place:livingroom
num_of_persons:1
person_id:Haeyoung1
face_rect:[125, 286, 198, 387]
full_rect:[49, 199, 301, 608]
behavior:sit down
predicate:
emotion:Neutral
epis

...

episode:1, scene:515, frame:1 st:['00:28:13;08'], et:['00:28:13;08']
episode:1, scene:515, frame:2 st:['00:28:13;08'], et:['00:28:13;08']
episode:1, scene:516, frame:0 st:['00:28:14;12'], et:['00:28:14;12']
place:
num_of_persons:1
person_id:Dokyung
face_rect:[402, 34, 741, 484]
full_rect:[216, 0, 964, 767]
behavior:none
predicate:
emotion:Anger
episode:1, scene:516, frame:1 st:['00:28:14;12'], et:['00:28:14;12']
place:
num_of_persons:1
person_id:Dokyung
face_rect:[269, 104, 608, 554]
full_rect:[75, 3, 823, 770]
behavior:none
predicate:
emotion:Anger
episode:1, scene:516, frame:2 st:['00:28:14;12'], et:['00:28:14;12']
place:
num_of_persons:2
person_id:Sangseok
face_rect:[700, 283, 997, 719]
full_rect:[619, 157, 1025, 762]
behavior:none
predicate:
emotion:Sadness
place:
num_of_persons:2
person_id:Dokyung
face_rect:[83, 244, 409, 678]
full_rect:[0, 3, 696, 769]
behavior:none
predicate:
emotion:Anger
episode:1, scene:516, frame:3 st:['00:28:14;12'], et:['00:28:14;12']
place:
num_of_persons

place:studio
num_of_persons:2
person_id:Gitae
face_rect:[283, 104, 528, 444]
full_rect:[80, 0, 661, 733]
behavior:none
predicate:
emotion:Happiness
place:studio
num_of_persons:2
person_id:Yijoon
face_rect:[705, 126, 935, 447]
full_rect:[581, 0, 1017, 723]
behavior:none
predicate:
emotion:Neutral
episode:1, scene:516, frame:29 st:['00:28:14;12'], et:['00:28:14;12']
place:studio
num_of_persons:2
person_id:Gitae
face_rect:[283, 104, 528, 444]
full_rect:[98, 0, 679, 733]
behavior:none
predicate:
emotion:Happiness
place:studio
num_of_persons:2
person_id:Yijoon
face_rect:[614, 123, 844, 444]
full_rect:[529, 0, 1011, 719]
behavior:none
predicate:
emotion:Neutral
episode:1, scene:516, frame:30 st:['00:28:14;12'], et:['00:28:14;12']
place:studio
num_of_persons:2
person_id:Gitae
face_rect:[283, 104, 528, 444]
full_rect:[98, 0, 679, 733]
behavior:none
predicate:
emotion:Happiness
place:studio
num_of_persons:2
person_id:Yijoon
face_rect:[577, 126, 807, 447]
full_rect:[529, 0, 1011, 719]
behavior:n

episode:1, scene:517, frame:26 st:['00:28:27;04'], et:['00:28:27;04']
episode:1, scene:517, frame:27 st:['00:28:27;04'], et:['00:28:27;04']
episode:1, scene:517, frame:28 st:['00:28:27;04'], et:['00:28:27;04']
episode:1, scene:518, frame:0 st:['00:28:36;16'], et:['00:28:36;16']
place:
num_of_persons:2
person_id:Dokyung
face_rect:[314, 77, 649, 574]
full_rect:[0, 0, 976, 768]
behavior:none
predicate:
emotion:Anger
place:
num_of_persons:2
person_id:Sangseok
face_rect:[682, 56, 1028, 633]
full_rect:[605, 0, 1027, 768]
behavior:none
predicate:
emotion:Surprise
episode:1, scene:518, frame:1 st:['00:28:36;16'], et:['00:28:36;16']
place:
num_of_persons:2
person_id:Dokyung
face_rect:[227, 118, 562, 615]
full_rect:[0, 0, 976, 768]
behavior:none
predicate:
emotion:Anger
place:
num_of_persons:2
person_id:Sangseok
face_rect:[664, 199, 961, 720]
full_rect:[605, 0, 1027, 768]
behavior:none
predicate:
emotion:Surprise
episode:1, scene:518, frame:2 st:['00:28:36;16'], et:['00:28:36;16']
place:
num_of_

place:studio
num_of_persons:1
person_id:Jiya
face_rect:[326, 127, 387, 215]
full_rect:[278, 93, 462, 741]
behavior:stand up
predicate:
emotion:Sadness
episode:1, scene:518, frame:31 st:['00:28:36;16'], et:['00:28:36;16']
place:studio
num_of_persons:1
person_id:Jiya
face_rect:[349, 102, 410, 190]
full_rect:[278, 93, 462, 741]
behavior:stand up
predicate:
emotion:Sadness
episode:1, scene:518, frame:32 st:['00:28:36;16'], et:['00:28:36;16']
place:studio
num_of_persons:1
person_id:Jiya
face_rect:[349, 102, 410, 190]
full_rect:[246, 41, 472, 742]
behavior:stand up
predicate:
emotion:Sadness
episode:1, scene:518, frame:33 st:['00:28:36;16'], et:['00:28:36;16']
place:studio
num_of_persons:1
person_id:Jiya
face_rect:[368, 78, 434, 190]
full_rect:[245, 41, 471, 742]
behavior:stand up
predicate:
emotion:Sadness
episode:1, scene:518, frame:34 st:['00:28:36;16'], et:['00:28:36;16']
place:studio
num_of_persons:1
person_id:Jiya
face_rect:[387, 93, 453, 205]
full_rect:[245, 41, 471, 742]
behavior:sta

place:studio
num_of_persons:1
person_id:Jiya
face_rect:[515, 202, 675, 421]
full_rect:[406, 31, 892, 770]
behavior:none
predicate:
emotion:Neutral
episode:1, scene:520, frame:7 st:['00:28:57;05'], et:['00:28:57;05']
place:studio
num_of_persons:1
person_id:Jiya
face_rect:[511, 198, 671, 417]
full_rect:[406, 31, 892, 770]
behavior:none
predicate:
emotion:Happiness
episode:1, scene:520, frame:8 st:['00:28:57;05'], et:['00:28:57;05']
place:studio
num_of_persons:1
person_id:Jiya
face_rect:[511, 194, 671, 413]
full_rect:[406, 31, 892, 770]
behavior:none
predicate:
emotion:Happiness
episode:1, scene:520, frame:9 st:['00:28:57;05'], et:['00:28:57;05']
place:studio
num_of_persons:1
person_id:Jiya
face_rect:[507, 195, 667, 414]
full_rect:[406, 31, 892, 770]
behavior:none
predicate:
emotion:Happiness
episode:1, scene:520, frame:10 st:['00:28:57;05'], et:['00:28:57;05']
place:studio
num_of_persons:1
person_id:Jiya
face_rect:[511, 202, 671, 421]
full_rect:[406, 31, 892, 770]
behavior:none
predicate

In [ ]:
{"directed": true, "multigraph": true, 
 "graph": {"name": "G", "node": {"fontsize": "7", "height": "0.1"}, 
           "edge": {"fontsize": "7"}}, 
 "nodes": [{"color": "lightgrey", "style": "filled", "id": "episode_1"}, 
           {"color": "lightgrey", "style": "filled", "id": "scene_10"}, 
           {"color": "lightgrey", "style": "filled", "id": "frame_0"}, 
           {"color": "lightblue", "style": "filled", "id": "kitchen"}, 
           {"color": "lightblue", "style": "filled", "id": "sound"}, 
           {"id": "Jeongsuk"}, 
           {"color": "green", "style": "filled", "id": "cook"}, 
           {"color": "blue", "style": "filled", "id": "Happiness"}, 
           {"id": "Deogi"}], 
 "links": [{"source": "Jeongsuk", "target": "cook", "key": 0}, 
           {"source": "Jeongsuk", "target": "Happiness", "key": 0}, 
           {"source": "Deogi", "target": "cook", "key": 0}, 
           {"source": "Deogi", "target": "Happiness", "key": 0}]
}